# roBERTa paragraph encodings

Encodes paragraphs with roberta for input into snorkel metal end model.

John Brandt


Last Updated: August 15, 2019

In [1]:
import torch
import ssl
import pandas as pd

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context

roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval()  # disable dropout (or leave in train mode to finetune)

Using cache found in /Users/john.brandt/.cache/torch/hub/pytorch_fairseq_master


loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz from cache at /Users/john.brandt/.cache/torch/pytorch_fairseq/83e3a689e28e5e4696ecb0bbb05a77355444a5c8a3437e0f736d8a564e80035e.c687083d14776c1979f3f71654febb42f2bb3d9a94ff7ebdfe1ac6748dba89d2
| dictionary: 50264 types


RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm(torch.Size([1024]), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (final_layer_norm): LayerNorm(torch.Size([1024]), eps=1e-05, elementwise_affine=True)
          )
          (1): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_f

## Encode sentences with roberta

In [3]:
df = pd.read_csv("../data/processed/gold_standard.csv")

In [ ]:
def encoder(sentence, max_len = 50):
    sentence_split = sentence.split(' ')
    sentence = ' '.join(sentence_split[:(min(len(sentence_split), max_len))])
    tokens = roberta.encode(sentence)
    tokens = tokens[:min(len(tokens), max_len)]
    feats = roberta.extract_features(tokens)
    if feats.shape[1] < max_len:
        padding = torch.zeros(1, max_len - feats.shape[1], 1024, dtype=torch.float)
        feats = torch.cat((feats, padding), 1)
    return feats

range_list = [x for x in range(len(df))]
encodings = [encoder(df['sentences'][x]) for x in range_list[:250]]

In [31]:
encodings_stacked = torch.cat(encodings, 0)
encodings_stacked = encodings_stacked.detach().numpy()
encodings_stacked.shape

(250, 50, 1024)

In [32]:
import numpy as np
np.save('../data/interim/encodings', encodings_stacked)